# Objetivos del test

En este test, crearás un script que sea capaz de leer los datos que se encuentran en varios ficheros json (en el directorio **datos**), aplicarles una transformación, y generar un fichero json de salida idéntico al fichero [resultado/result.json](./resultado/result.json).

El json generado debe contener una copia del objeto `subscriptions` del fichero de entrada [datos/manifest.json](./datos/manifest.json), pero con algunas modificaciones:

- Se añadirá a cada `subscription` un atributo `description` calculado a partir de varias propiedades de la subscripción y la vertical.
- Se reemplazará el valor de los atributos `notification.http.url` y `notification.httpCustom.url` por URLs configuradas en el fichero [datos/entorno/environment.json](./datos/entorno/environment.json)
- Se reemplazará cualquier ocurrencia del patrón `${variable}` en campos de tipo texto, por el valor de dicha variable en el fichero [datos/deployment.json](./datos/deployment.json)

El schema de los tres ficheros de entrada está descrito formalmente en los ficheros del directorio **schema**:

- [schema/manifest.cue](./schema/manifest.cue): descriptor del formato del fichero [datos/vertical.json](./datos/vertical.json)
- [schema/deployment.cue](./schema/deployment.cue): descriptor del formato del fichero [datos/deployment.json](./datos/deployment.json)
- [schema/environment.cue](./schema/environment.cue): descriptor del formato del fichero [datos/entorno/environment.json](./datos/entorno/environment.json)

## Pruebas Python

### Prueba 1

Edita el código de la celda de abajo para implementar la función `read_deployment`. Esta función debe leer el fichero cuyo nombre se le pase como parámetro, e interpretar su contenido como un objeto json.

In [65]:
import json

from typing import Any

def read_deployment(path: str) -> Any:
    pass

read_deployment("datos/deployment.json")

{'deployment': {'subservice': '/alumbrado',
  'variables': {'servicepath_commands': '/lightningcommands',
   'lastdata_duration': '20 days'},
  'sources': {'manifests': {'files': ['manifest.json']},
   'environment': {'path': 'entorno', 'files': ['environment.json']}}}}

### Prueba 2

Edita el código de la celda de abajo para implementar la función `collect_sources`. Esta función debe recibir la ruta al fichero "deployment.json", y ejecutar las siguientes tareas:

- Leer el fichero `deployment.json` usando la función anterior `read_deployment`.
- Leer todos los ficheros referenciados por el elemento `sources` del `deployment`, interpretando sus contenidos como objetos json.
- Unificar todos los objetos json leídos en un único `dict`, usando `dict.update`
- Devolver el `dict` unificado

In [66]:
import os.path

def collect_sources(path: str) -> Any:
    pass

collect_sources("datos/deployment.json")

{'deployment': {'subservice': '/alumbrado',
  'variables': {'servicepath_commands': '/lightningcommands',
   'lastdata_duration': '20 days'},
  'sources': {'manifests': {'files': ['manifest.json']},
   'environment': {'path': 'entorno', 'files': ['environment.json']}}},
 'meta': {'name': 'streetlight', 'version': '1.9.0'},
 'ddls': {'sources': {'base': {'path': 'model/db',
    'files': ['ddls.sql'],
    'weight': 0}},
  'variables': {'duration': '${lastdata_duration}'}},
 'subscriptions': {'streetlight-historic': {'status': 'active',
   'subject': {'entities': [{'idPattern': '.*', 'type': 'Streetlight'}],
    'condition': {'attrs': ['TimeInstant']}},
   'notification': {'http': {'url': 'HISTORIC'},
    'attrs': ['TimeInstant',
     'activePower',
     'address',
     'controllingMethod',
     'dateLastSwitchingOff',
     'dateLastSwitchingOn',
     'dateMeteringStarted',
     'dateServiceStarted',
     'delay'],
    'onlyChangedAttrs': False,
    'attrsFormat': 'normalized'}},
  'stree

### Prueba 3

Edita el código de la celda de abajo para implementar la función `update_subscriptions`. Esta función debe recibir el resultado de la función `collect_sources`, recorrer el diccionario de `subscriptions`, ejecutar las siguientes modificaciones, y devolver el diccionario de `subscriptions` modificado:

- Añadir a cada `subscription` una nueva propiedad `description` resultado de llamar a la funcion `sub_description` con el resultado de `collect_sources`, y el nombre de la suscripción.
- Modificar el atributo `url` de la suscripción, para sustituirlo por el valor correspondiente del diccionario `["environment"]["notificationEndpoints"]` (por ejemplo, si el atributo `["notification"]["http"]["url"]` de la suscripción es *LASTDATA*, habría que cambiarlo por *http://iot-cygnus:5059/notify*.

In [67]:
def sub_description(sources: Any, sub_name: str) -> str:
    return "DEPLOYER:" + sources["meta"]["name"] + ":" + sources["meta"]["version"] + ":" + sub_name

def update_subscriptions(sources: Any):
    pass

sources = collect_sources("datos/deployment.json")
subs    = update_subscriptions(sources)
subs['streetlight-commands-historic']

{'status': 'active',
 'subject': {'entities': [{'idPattern': '.*', 'type': 'Streetlight'}],
  'condition': {'attrs': ['commandTimeinstant'],
   'expression': {'q': "enableHistoricCommand:'true'"}}},
 'notification': {'httpCustom': {'url': 'http://iot-cygnus:5057/notify',
   'headers': {'Fiware-Servicepath': '${servicepath_commands}'}},
  'attrs': ['setIlluminanceLevel',
   'setIlluminanceLevel_info',
   'setIlluminanceLevel_status',
   'dateLastCommanded',
   'lastCommander',
   'powerState',
   'workingMode',
   'commandTimeinstant'],
  'onlyChangedAttrs': False,
  'attrsFormat': 'normalized'},
 'description': 'DEPLOYER:streetlight:1.9.0:streetlight-commands-historic'}

### Prueba 4

Edita el código de la celda de abajo para implementar la función `replace_variables`. Esta función debe recibir un diccionario de datos y otro de variables, y debe recorrer el diccionario de datos recursivamente reemplazando el patrón *${variable}* en cualquier texto por la correspondiente variable del diccionario de variables.

Por ejemplo, `replace_variables({"a":{"b":"test vale ${test}"}}, {"test": "valor"})` debe devolver el resultado `{"a":{"b":"test vale valor"}}`

In [68]:
from typing import Dict
import string

def replace_variables(data: Any, variables: Dict[str, str]) -> Any:
    pass

replaced = replace_variables(subs, sources['deployment']['variables'])
replaced['streetlight-commands-historic']

{'status': 'active',
 'subject': {'entities': [{'idPattern': '.*', 'type': 'Streetlight'}],
  'condition': {'attrs': ['commandTimeinstant'],
   'expression': {'q': "enableHistoricCommand:'true'"}}},
 'notification': {'httpCustom': {'url': 'http://iot-cygnus:5057/notify',
   'headers': {'Fiware-Servicepath': '/lightningcommands'}},
  'attrs': ['setIlluminanceLevel',
   'setIlluminanceLevel_info',
   'setIlluminanceLevel_status',
   'dateLastCommanded',
   'lastCommander',
   'powerState',
   'workingMode',
   'commandTimeinstant'],
  'onlyChangedAttrs': False,
  'attrsFormat': 'normalized'},
 'description': 'DEPLOYER:streetlight:1.9.0:streetlight-commands-historic'}

### Prueba 5

Aseguráte de que tu código supera los tests de la celda de abajo. No debes modificar los tests.

In [69]:
import unittest

class Test(unittest.TestCase):

    def test_read_deployment(self):
        data = read_deployment("datos/deployment.json")
        self.maxDiff = None
        self.assertEqual(json.dumps(data, sort_keys=True), """{"deployment": {"sources": {"environment": {"files": ["environment.json"], "path": "entorno"}, "manifests": {"files": ["manifest.json"]}}, "subservice": "/alumbrado", "variables": {"lastdata_duration": "20 days", "servicepath_commands": "/lightningcommands"}}}""")

    def test_collect_sources(self):
        data = collect_sources("datos/deployment.json")
        self.maxDiff = None
        self.assertEqual(json.dumps(data, sort_keys=True), """{"ddls": {"sources": {"base": {"files": ["ddls.sql"], "path": "model/db", "weight": 0}}, "variables": {"duration": "${lastdata_duration}"}}, "deployment": {"sources": {"environment": {"files": ["environment.json"], "path": "entorno"}, "manifests": {"files": ["manifest.json"]}}, "subservice": "/alumbrado", "variables": {"lastdata_duration": "20 days", "servicepath_commands": "/lightningcommands"}}, "environment": {"api": {"keystone": "http://iot.lab.urbo2.es:5001", "orion": "https://iot.lab.urbo2.es:1026", "postgis": "host=iot.lab.urbo2.es port=5432"}, "database": "urbodeployer", "databaseSchemas": {"DEFAULT": "urbodeployer"}, "notificationEndpoints": {"HISTORIC": "http://iot-cygnus:5057/notify", "LASTDATA": "http://iot-cygnus:5059/notify", "RULES": "https://iot-perseo:9090/notices"}, "service": "urbodeployer"}, "meta": {"name": "streetlight", "version": "1.9.0"}, "subscriptions": {"streetlight-cep": {"notification": {"attrs": [], "attrsFormat": "normalized", "http": {"url": "RULES"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": []}, "entities": [{"idPattern": ".*", "type": "Streetlight"}]}}, "streetlight-commands-historic": {"notification": {"attrs": ["setIlluminanceLevel", "setIlluminanceLevel_info", "setIlluminanceLevel_status", "dateLastCommanded", "lastCommander", "powerState", "workingMode", "commandTimeinstant"], "attrsFormat": "normalized", "httpCustom": {"headers": {"Fiware-Servicepath": "${servicepath_commands}"}, "url": "HISTORIC"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["commandTimeinstant"], "expression": {"q": "enableHistoricCommand:'true'"}}, "entities": [{"idPattern": ".*", "type": "Streetlight"}]}}, "streetlight-historic": {"notification": {"attrs": ["TimeInstant", "activePower", "address", "controllingMethod", "dateLastSwitchingOff", "dateLastSwitchingOn", "dateMeteringStarted", "dateServiceStarted", "delay"], "attrsFormat": "normalized", "http": {"url": "HISTORIC"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["TimeInstant"]}, "entities": [{"idPattern": ".*", "type": "Streetlight"}]}}, "streetlight-lastdata": {"notification": {"attrs": ["TimeInstant", "activePower", "address", "controllingMethod", "dateLastSwitchingOff", "dateLastSwitchingOn", "dateMeteringStarted", "dateServiceStarted", "delay"], "attrsFormat": "normalized", "http": {"url": "LASTDATA"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["TimeInstant"]}, "entities": [{"idPattern": ".*", "type": "Streetlight"}]}}, "streetlightcontrolcabinet-cep-cmdilluminancelevel": {"notification": {"attrs": ["cmdIlluminanceLevel"], "attrsFormat": "normalized", "http": {"url": "RULES"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["cmdIlluminanceLevel"]}, "entities": [{"idPattern": ".*", "type": "StreetlightControlCabinet"}]}}, "streetlightcontrolcabinet-cep-cmdpowerstate": {"notification": {"attrs": ["cmdPowerState"], "attrsFormat": "normalized", "http": {"url": "RULES"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["cmdPowerState"]}, "entities": [{"idPattern": ".*", "type": "StreetlightControlCabinet"}]}}, "streetlightcontrolcabinet-cep-cmdworkingmode": {"notification": {"attrs": ["cmdWorkingMode"], "attrsFormat": "normalized", "http": {"url": "RULES"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["cmdWorkingMode"]}, "entities": [{"idPattern": ".*", "type": "StreetlightControlCabinet"}]}}, "streetlightcontrolcabinet-historic": {"notification": {"attrs": ["TimeInstant", "activePowerR", "activePowerS", "activePowerT", "address", "brandName", "cosPhi", "dateLastProgramming", "dateLastSwitchingOff", "dateLastSwitchingOn", "dateMeteringStarted", "dateServiceStarted", "energyConsumed", "energyCost"], "attrsFormat": "normalized", "http": {"url": "HISTORIC"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["TimeInstant"]}, "entities": [{"idPattern": ".*", "type": "StreetlightControlCabinet"}]}}, "streetlightcontrolcabinet-lastdata": {"notification": {"attrs": ["TimeInstant", "activePowerR", "activePowerS", "activePowerT", "address", "brandName", "cosPhi", "dateLastProgramming", "dateLastSwitchingOff", "dateLastSwitchingOn", "dateMeteringStarted", "dateServiceStarted", "energyConsumed", "energyCost", "frequency"], "attrsFormat": "normalized", "http": {"url": "LASTDATA"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["TimeInstant"]}, "entities": [{"idPattern": ".*", "type": "StreetlightControlCabinet"}]}}, "streetlightgroup-cep-cmdilluminancelevel": {"notification": {"attrs": ["cmdIlluminanceLevel"], "attrsFormat": "normalized", "http": {"url": "RULES"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["cmdIlluminanceLevel"]}, "entities": [{"idPattern": ".*", "type": "StreetlightGroup"}]}}, "streetlightzone-cep-cmdilluminancelevel": {"notification": {"attrs": ["cmdIlluminanceLevel"], "attrsFormat": "normalized", "http": {"url": "RULES"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["cmdIlluminanceLevel"]}, "entities": [{"idPattern": ".*", "type": "StreetlightZone"}]}}, "streetlightzone-cep-cmdworkingmode": {"notification": {"attrs": ["cmdWorkingMode"], "attrsFormat": "normalized", "http": {"url": "RULES"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["cmdWorkingMode"]}, "entities": [{"idPattern": ".*", "type": "StreetlightZone"}]}}}}""")

    def test_update_subscriptions(self):
        data = update_subscriptions(collect_sources("datos/deployment.json"))
        self.maxDiff = None
        self.assertEqual(json.dumps(data, sort_keys=True), """{"streetlight-cep": {"description": "DEPLOYER:streetlight:1.9.0:streetlight-cep", "notification": {"attrs": [], "attrsFormat": "normalized", "http": {"url": "https://iot-perseo:9090/notices"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": []}, "entities": [{"idPattern": ".*", "type": "Streetlight"}]}}, "streetlight-commands-historic": {"description": "DEPLOYER:streetlight:1.9.0:streetlight-commands-historic", "notification": {"attrs": ["setIlluminanceLevel", "setIlluminanceLevel_info", "setIlluminanceLevel_status", "dateLastCommanded", "lastCommander", "powerState", "workingMode", "commandTimeinstant"], "attrsFormat": "normalized", "httpCustom": {"headers": {"Fiware-Servicepath": "${servicepath_commands}"}, "url": "http://iot-cygnus:5057/notify"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["commandTimeinstant"], "expression": {"q": "enableHistoricCommand:'true'"}}, "entities": [{"idPattern": ".*", "type": "Streetlight"}]}}, "streetlight-historic": {"description": "DEPLOYER:streetlight:1.9.0:streetlight-historic", "notification": {"attrs": ["TimeInstant", "activePower", "address", "controllingMethod", "dateLastSwitchingOff", "dateLastSwitchingOn", "dateMeteringStarted", "dateServiceStarted", "delay"], "attrsFormat": "normalized", "http": {"url": "http://iot-cygnus:5057/notify"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["TimeInstant"]}, "entities": [{"idPattern": ".*", "type": "Streetlight"}]}}, "streetlight-lastdata": {"description": "DEPLOYER:streetlight:1.9.0:streetlight-lastdata", "notification": {"attrs": ["TimeInstant", "activePower", "address", "controllingMethod", "dateLastSwitchingOff", "dateLastSwitchingOn", "dateMeteringStarted", "dateServiceStarted", "delay"], "attrsFormat": "normalized", "http": {"url": "http://iot-cygnus:5059/notify"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["TimeInstant"]}, "entities": [{"idPattern": ".*", "type": "Streetlight"}]}}, "streetlightcontrolcabinet-cep-cmdilluminancelevel": {"description": "DEPLOYER:streetlight:1.9.0:streetlightcontrolcabinet-cep-cmdilluminancelevel", "notification": {"attrs": ["cmdIlluminanceLevel"], "attrsFormat": "normalized", "http": {"url": "https://iot-perseo:9090/notices"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["cmdIlluminanceLevel"]}, "entities": [{"idPattern": ".*", "type": "StreetlightControlCabinet"}]}}, "streetlightcontrolcabinet-cep-cmdpowerstate": {"description": "DEPLOYER:streetlight:1.9.0:streetlightcontrolcabinet-cep-cmdpowerstate", "notification": {"attrs": ["cmdPowerState"], "attrsFormat": "normalized", "http": {"url": "https://iot-perseo:9090/notices"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["cmdPowerState"]}, "entities": [{"idPattern": ".*", "type": "StreetlightControlCabinet"}]}}, "streetlightcontrolcabinet-cep-cmdworkingmode": {"description": "DEPLOYER:streetlight:1.9.0:streetlightcontrolcabinet-cep-cmdworkingmode", "notification": {"attrs": ["cmdWorkingMode"], "attrsFormat": "normalized", "http": {"url": "https://iot-perseo:9090/notices"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["cmdWorkingMode"]}, "entities": [{"idPattern": ".*", "type": "StreetlightControlCabinet"}]}}, "streetlightcontrolcabinet-historic": {"description": "DEPLOYER:streetlight:1.9.0:streetlightcontrolcabinet-historic", "notification": {"attrs": ["TimeInstant", "activePowerR", "activePowerS", "activePowerT", "address", "brandName", "cosPhi", "dateLastProgramming", "dateLastSwitchingOff", "dateLastSwitchingOn", "dateMeteringStarted", "dateServiceStarted", "energyConsumed", "energyCost"], "attrsFormat": "normalized", "http": {"url": "http://iot-cygnus:5057/notify"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["TimeInstant"]}, "entities": [{"idPattern": ".*", "type": "StreetlightControlCabinet"}]}}, "streetlightcontrolcabinet-lastdata": {"description": "DEPLOYER:streetlight:1.9.0:streetlightcontrolcabinet-lastdata", "notification": {"attrs": ["TimeInstant", "activePowerR", "activePowerS", "activePowerT", "address", "brandName", "cosPhi", "dateLastProgramming", "dateLastSwitchingOff", "dateLastSwitchingOn", "dateMeteringStarted", "dateServiceStarted", "energyConsumed", "energyCost", "frequency"], "attrsFormat": "normalized", "http": {"url": "http://iot-cygnus:5059/notify"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["TimeInstant"]}, "entities": [{"idPattern": ".*", "type": "StreetlightControlCabinet"}]}}, "streetlightgroup-cep-cmdilluminancelevel": {"description": "DEPLOYER:streetlight:1.9.0:streetlightgroup-cep-cmdilluminancelevel", "notification": {"attrs": ["cmdIlluminanceLevel"], "attrsFormat": "normalized", "http": {"url": "https://iot-perseo:9090/notices"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["cmdIlluminanceLevel"]}, "entities": [{"idPattern": ".*", "type": "StreetlightGroup"}]}}, "streetlightzone-cep-cmdilluminancelevel": {"description": "DEPLOYER:streetlight:1.9.0:streetlightzone-cep-cmdilluminancelevel", "notification": {"attrs": ["cmdIlluminanceLevel"], "attrsFormat": "normalized", "http": {"url": "https://iot-perseo:9090/notices"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["cmdIlluminanceLevel"]}, "entities": [{"idPattern": ".*", "type": "StreetlightZone"}]}}, "streetlightzone-cep-cmdworkingmode": {"description": "DEPLOYER:streetlight:1.9.0:streetlightzone-cep-cmdworkingmode", "notification": {"attrs": ["cmdWorkingMode"], "attrsFormat": "normalized", "http": {"url": "https://iot-perseo:9090/notices"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["cmdWorkingMode"]}, "entities": [{"idPattern": ".*", "type": "StreetlightZone"}]}}}""")

    def test_replace_subscriptions(self):
        sources  = collect_sources("datos/deployment.json")
        subs     = update_subscriptions(sources)
        replaced = replace_variables(subs, sources['deployment']['variables'])
        self.maxDiff = None
        self.assertEqual(json.dumps(replaced, sort_keys=True), """{"streetlight-cep": {"description": "DEPLOYER:streetlight:1.9.0:streetlight-cep", "notification": {"attrs": [], "attrsFormat": "normalized", "http": {"url": "https://iot-perseo:9090/notices"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": []}, "entities": [{"idPattern": ".*", "type": "Streetlight"}]}}, "streetlight-commands-historic": {"description": "DEPLOYER:streetlight:1.9.0:streetlight-commands-historic", "notification": {"attrs": ["setIlluminanceLevel", "setIlluminanceLevel_info", "setIlluminanceLevel_status", "dateLastCommanded", "lastCommander", "powerState", "workingMode", "commandTimeinstant"], "attrsFormat": "normalized", "httpCustom": {"headers": {"Fiware-Servicepath": "/lightningcommands"}, "url": "http://iot-cygnus:5057/notify"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["commandTimeinstant"], "expression": {"q": "enableHistoricCommand:'true'"}}, "entities": [{"idPattern": ".*", "type": "Streetlight"}]}}, "streetlight-historic": {"description": "DEPLOYER:streetlight:1.9.0:streetlight-historic", "notification": {"attrs": ["TimeInstant", "activePower", "address", "controllingMethod", "dateLastSwitchingOff", "dateLastSwitchingOn", "dateMeteringStarted", "dateServiceStarted", "delay"], "attrsFormat": "normalized", "http": {"url": "http://iot-cygnus:5057/notify"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["TimeInstant"]}, "entities": [{"idPattern": ".*", "type": "Streetlight"}]}}, "streetlight-lastdata": {"description": "DEPLOYER:streetlight:1.9.0:streetlight-lastdata", "notification": {"attrs": ["TimeInstant", "activePower", "address", "controllingMethod", "dateLastSwitchingOff", "dateLastSwitchingOn", "dateMeteringStarted", "dateServiceStarted", "delay"], "attrsFormat": "normalized", "http": {"url": "http://iot-cygnus:5059/notify"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["TimeInstant"]}, "entities": [{"idPattern": ".*", "type": "Streetlight"}]}}, "streetlightcontrolcabinet-cep-cmdilluminancelevel": {"description": "DEPLOYER:streetlight:1.9.0:streetlightcontrolcabinet-cep-cmdilluminancelevel", "notification": {"attrs": ["cmdIlluminanceLevel"], "attrsFormat": "normalized", "http": {"url": "https://iot-perseo:9090/notices"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["cmdIlluminanceLevel"]}, "entities": [{"idPattern": ".*", "type": "StreetlightControlCabinet"}]}}, "streetlightcontrolcabinet-cep-cmdpowerstate": {"description": "DEPLOYER:streetlight:1.9.0:streetlightcontrolcabinet-cep-cmdpowerstate", "notification": {"attrs": ["cmdPowerState"], "attrsFormat": "normalized", "http": {"url": "https://iot-perseo:9090/notices"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["cmdPowerState"]}, "entities": [{"idPattern": ".*", "type": "StreetlightControlCabinet"}]}}, "streetlightcontrolcabinet-cep-cmdworkingmode": {"description": "DEPLOYER:streetlight:1.9.0:streetlightcontrolcabinet-cep-cmdworkingmode", "notification": {"attrs": ["cmdWorkingMode"], "attrsFormat": "normalized", "http": {"url": "https://iot-perseo:9090/notices"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["cmdWorkingMode"]}, "entities": [{"idPattern": ".*", "type": "StreetlightControlCabinet"}]}}, "streetlightcontrolcabinet-historic": {"description": "DEPLOYER:streetlight:1.9.0:streetlightcontrolcabinet-historic", "notification": {"attrs": ["TimeInstant", "activePowerR", "activePowerS", "activePowerT", "address", "brandName", "cosPhi", "dateLastProgramming", "dateLastSwitchingOff", "dateLastSwitchingOn", "dateMeteringStarted", "dateServiceStarted", "energyConsumed", "energyCost"], "attrsFormat": "normalized", "http": {"url": "http://iot-cygnus:5057/notify"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["TimeInstant"]}, "entities": [{"idPattern": ".*", "type": "StreetlightControlCabinet"}]}}, "streetlightcontrolcabinet-lastdata": {"description": "DEPLOYER:streetlight:1.9.0:streetlightcontrolcabinet-lastdata", "notification": {"attrs": ["TimeInstant", "activePowerR", "activePowerS", "activePowerT", "address", "brandName", "cosPhi", "dateLastProgramming", "dateLastSwitchingOff", "dateLastSwitchingOn", "dateMeteringStarted", "dateServiceStarted", "energyConsumed", "energyCost", "frequency"], "attrsFormat": "normalized", "http": {"url": "http://iot-cygnus:5059/notify"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["TimeInstant"]}, "entities": [{"idPattern": ".*", "type": "StreetlightControlCabinet"}]}}, "streetlightgroup-cep-cmdilluminancelevel": {"description": "DEPLOYER:streetlight:1.9.0:streetlightgroup-cep-cmdilluminancelevel", "notification": {"attrs": ["cmdIlluminanceLevel"], "attrsFormat": "normalized", "http": {"url": "https://iot-perseo:9090/notices"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["cmdIlluminanceLevel"]}, "entities": [{"idPattern": ".*", "type": "StreetlightGroup"}]}}, "streetlightzone-cep-cmdilluminancelevel": {"description": "DEPLOYER:streetlight:1.9.0:streetlightzone-cep-cmdilluminancelevel", "notification": {"attrs": ["cmdIlluminanceLevel"], "attrsFormat": "normalized", "http": {"url": "https://iot-perseo:9090/notices"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["cmdIlluminanceLevel"]}, "entities": [{"idPattern": ".*", "type": "StreetlightZone"}]}}, "streetlightzone-cep-cmdworkingmode": {"description": "DEPLOYER:streetlight:1.9.0:streetlightzone-cep-cmdworkingmode", "notification": {"attrs": ["cmdWorkingMode"], "attrsFormat": "normalized", "http": {"url": "https://iot-perseo:9090/notices"}, "onlyChangedAttrs": false}, "status": "active", "subject": {"condition": {"attrs": ["cmdWorkingMode"]}, "entities": [{"idPattern": ".*", "type": "StreetlightZone"}]}}}""")

unittest.main(argv=[''], exit=False)

....
----------------------------------------------------------------------
Ran 4 tests in 0.008s

OK


## Pruebas CUE

El objetivo de esta prueba es crear `schema`s (descripciones formales) que puedan servir para validar los ficheros json con los que has estado trabajando. Como lenguaje para describir el schema, vamos a utilizar **cue** (https://cuelang.org/).

`cue` es un lenguaje reciente y es muy posible que no hayas trabajado nunca con él. Puedes familizarizarte con su uso con los tutoriales que encontrarás aquí: https://cuetorials.com/

Asegúrate de descargar e instalar en tu entorno linux/unix la versión 0.4.3 de cue, que podrás obtener aquí: https://github.com/cue-lang/cue/releases/tag/v0.4.3

### Prueba 1

Modifica el fichero [schema/deployment.cue](schema/deployment.cue) para que el comando `cue vet schema/deployment.cue datos/deployment.json` no devuelva ningún error.

In [70]:
# (Debes conseguir que desaparezca el error)

!cue vet -v schema/deployment.cue datos/deployment.json

deployment.variables.lastdata_duration: conflicting values "20 days" and int (mismatched types string and int):
    ./datos/deployment.json:6:34
    ./schema/deployment.cue:14:19
deployment.variables.servicepath_commands: conflicting values "/lightningcommands" and int (mismatched types string and int):
    ./datos/deployment.json:5:37
    ./schema/deployment.cue:14:19


### Prueba 2

Modifica el fichero [schema/environment.cue](schema/environment.cue) para que el comando `cue vet schema/environment.cue datos/entorno/environment.json` no devuelva ningún error.

In [71]:
# (Debes conseguir que desaparezca el error)

!cue vet -v schema/environment.cue datos/entorno/environment.json

environment.api.orion: invalid value "https://iot.lab.urbo2.es:1026" (out of bound =~"http://.*"):
    ./schema/environment.cue:11:24
    ./datos/entorno/environment.json:10:25
    ./schema/environment.cue:11:15
environment.notificationEndpoints.RULES: invalid value "https://iot-perseo:9090/notices" (out of bound =~"http://.*"):
    ./schema/environment.cue:19:21
    ./datos/entorno/environment.json:16:22
    ./schema/environment.cue:19:12
    ./schema/environment.cue:23:36


### Prueba 3

Modifica el fichero [schema/manifest.cue](schema/manifest.cue) para que el comando `cue vet schema/manifest.cue datos/manifest.json` no devuelva ningún error.


In [72]:
# (Debes conseguir que desaparezca el error)

!cue vet -v schema/manifest.cue datos/manifest.json

subscriptions."streetlight-commands-historic".notification.httpCustom: field not allowed: headers:
    ./datos/manifest.json:141:11
    ./schema/manifest.cue:81:18
    ./schema/manifest.cue:88:13
